
# Análisis de Secuencias Nucleotídicas de Integronas de Shewanella

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [1]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display

def instalar_librerias(b=""):
    with output:
        librerias = [
            "import_ipynb",
            "nbconvert",
            "ipywidgets",  # Asegúrate de que ya esté instalado
            "biopython",
            "pandas",
            "numpy",
            "matplotlib",
            "tqdm",
            "odfpy",
            "requests",
            "gdown",
            "mafft",
            "clustalo",
            "muscle",
            "fasttree"
        ]

        barra_progreso = widgets.IntProgress(
            value=0,
            min=0,
            max=len(librerias),
            bar_style='info',
            orientation='horizontal'
        )

        etiqueta = widgets.Label(value='Iniciando instalación...')
        display(widgets.VBox([etiqueta, barra_progreso]))

        for libreria in librerias:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
            barra_progreso.value += 1

        etiqueta.value = 'Instalación completada.'

# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)
instalar_librerias()

Output()

In [2]:
def combinar_en_multifasta(rutas_archivos_fasta, archivo_multifasta):
    with open(archivo_multifasta, 'w') as archivo_salida:
        for ruta_fasta in rutas_archivos_fasta:
            with open(ruta_fasta, 'r') as archivo_entrada:
                archivo_salida.write(archivo_entrada.read() + "\n")

# Combina los archivos FASTA en un solo archivo multifasta
#archivo_multifasta = "secuencias_combinadas.fasta"
#combinar_en_multifasta(rutas_archivos_fasta, archivo_multifasta)


In [3]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO
import os

In [4]:
def alinear_con_mafft_rapido(archivo_multifasta):
    from Bio.Align.Applications import MafftCommandline
    mafft_cline = MafftCommandline(input=archivo_multifasta, auto=True)
    stdout, stderr = mafft_cline()
    with open("alineamiento_mafft_rapido.fasta", "w") as handle:
        handle.write(stdout)
    return "alineamiento_mafft_rapido.fasta"

# Alinea las secuencias usando la configuración rápida de MAFFT
#ruta_alineamiento_mafft_rapido = alinear_con_mafft_rapido(archivo_multifasta)


In [5]:
from Bio import SeqIO
from Bio.Align.Applications import MafftCommandline
import multiprocessing
from ipywidgets import IntProgress, VBox, Label
from IPython.display import display
import os

def dividir_archivo_multifasta(archivo_multifasta, num_secuencias_por_archivo):
    with open(archivo_multifasta, "r") as handle:
        secuencias = list(SeqIO.parse(handle, "fasta"))

    archivos_divididos = []
    for i in range(0, len(secuencias), num_secuencias_por_archivo):
        archivo_parte = f"parte_{i//num_secuencias_por_archivo}.fasta"
        with open(archivo_parte, "w") as output_handle:
            SeqIO.write(secuencias[i:i + num_secuencias_por_archivo], output_handle, "fasta")
        archivos_divididos.append(archivo_parte)

    return archivos_divididos

def alinear_con_mafft(archivo_fasta):
    archivo_salida = archivo_fasta.replace(".fasta", "_alineado.fasta")
    mafft_cline = MafftCommandline(input=archivo_fasta, auto=True)
    stdout, stderr = mafft_cline()
    with open(archivo_salida, "w") as output_handle:
        output_handle.write(stdout)
    return archivo_salida

def alinear_archivos_en_paralelo(archivos_divididos):
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    resultados = pool.map(alinear_con_mafft, archivos_divididos)
    pool.close()
    pool.join()
    return resultados

def alinear_secuencias(archivo_multifasta, num_secuencias_por_archivo):
    archivos_divididos = dividir_archivo_multifasta(archivo_multifasta, num_secuencias_por_archivo)

    # Barra de progreso
    progress_bar = IntProgress(min=0, max=len(archivos_divididos))
    progress_label = Label('Iniciando alineamiento...')
    display(VBox([progress_label, progress_bar]))

    # Alineamiento en paralelo
    for i, _ in enumerate(alinear_archivos_en_paralelo(archivos_divididos), 1):
        progress_bar.value = i
        progress_label.value = f"Alineando archivo {i} de {len(archivos_divididos)}"

    progress_label.value = 'Alineamiento completado.'

# Uso de la función
archivo_multifasta = "tu_archivo_combinado.fasta"  # Reemplaza con la ruta de tu archivo
num_secuencias_por_archivo = 100  # Ajusta según tus necesidades
#alinear_secuencias(archivo_multifasta, num_secuencias_por_archivo)



In [6]:
from Bio import SeqIO
from Bio.Align.Applications import MafftCommandline
import os
import multiprocessing

def dividir_archivo_multifasta2(archivo_multifasta, num_secuencias_por_archivo):
    with open(archivo_multifasta, "r") as handle:
        secuencias = list(SeqIO.parse(handle, "fasta"))

    archivos_divididos = []
    for i in range(0, len(secuencias), num_secuencias_por_archivo):
        parte = secuencias[i:i + num_secuencias_por_archivo]
        archivo_parte = f"parte_{i//num_secuencias_por_archivo}.fasta"
        archivos_divididos.append(archivo_parte)
        with open(archivo_parte, "w") as output_handle:
            SeqIO.write(parte, output_handle, "fasta")

    return archivos_divididos

def alinear_con_mafft2(archivo_fasta):
    archivo_salida = archivo_fasta.replace(".fasta", "_alineado.fasta")
    # Ajustar parámetros de MAFFT para manejar mejor secuencias largas
    mafft_cline = MafftCommandline(input=archivo_fasta, auto=True, op=1.53, maxiterate=1000, fft=False)
    try:
        stdout, stderr = mafft_cline()
        with open(archivo_salida, "w") as output_handle:
            output_handle.write(stdout)
        return archivo_salida
    except Exception as e:
        print(f"Error al alinear {archivo_fasta}: {e}")
        return None

def alinear_archivo2(archivo):
    print(f"Alineando {archivo} con MAFFT...")
    archivo_alineado = alinear_con_mafft(archivo)
    if archivo_alineado:
        print(f"Archivo alineado guardado en {archivo_alineado}")

# Ajusta este valor según tus necesidades y recursos disponibles
num_secuencias_por_archivo = 4

# Dividir archivo multifasta
#archivos_divididos = dividir_archivo_multifasta("secuencias_combinadas.fasta", num_secuencias_por_archivo)

# Utilizar paralelización para acelerar el proceso de alineamiento
def alinear_combinaciones2(archivos_divididos):
  pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
  pool.map(alinear_archivo2, archivos_divididos)
  pool.close()
  pool.join()
#alinear_combinaciones2(archivos_divididos)

In [7]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO
from Bio import Phylo
from Bio.Phylo.Applications import FastTreeCommandline
import os
import subprocess

def construir_arbol_con_mafft(ruta_alineamiento):
    # Verificar si el archivo de alineamiento existe
    if not os.path.exists(ruta_alineamiento):
        print(f"El archivo {ruta_alineamiento} no existe.")
        return

    archivo_arbol = "arbol_mafft.nwk"

    try:
        # Ejecutar FastTree
        fasttree_cline = FastTreeCommandline(input=ruta_alineamiento, out=archivo_arbol)
        subprocess.run(str(fasttree_cline), shell=True, check=True)

        # Leer y dibujar el árbol
        arbol = Phylo.read(archivo_arbol, "newick")
        Phylo.draw(arbol)
    except Exception as e:
        print(f"Ocurrió un error al construir el árbol: {e}")

# Construcción del árbol
#construir_arbol_con_mafft("/content/parte_4_alineado.fasta")
